# Homework 10
#### Course Notes
**Language Models:** https://github.com/rjenki/BIOS512/tree/main/lecture17  
**Unix:** https://github.com/rjenki/BIOS512/tree/main/lecture18  
**Docker:** https://github.com/rjenki/BIOS512/tree/main/lecture19

## Question 1
#### Make a language model that uses ngrams and allows the user to specify start words, but uses a random start if one is not specified.

#### a) Make a function to tokenize the text.

#### b) Make a function generate keys for ngrams.

#### c) Make a function to build an ngram table.

#### d) Function to digest the text.

#### e) Function to digest the url.

#### f) Function that gives random start.

#### g) Function to predict the next word.

#### h) Function that puts everything together. Specify that if the user does not give a start word, then the random start will be used.

In [33]:
generate_ngram_text <- function(text, n = 3, start = NULL, length = 50) {
  tokens <- tolower(text)
  tokens <- gsub("[^a-z0-9'\\s]", "", tokens)
  tokens <- unlist(strsplit(tokens, "\\s+"))
  tokens <- tokens[tokens != ""]
  
  if (length(tokens) <= n) {
    stop("Text is too short to generate n-grams with n = ", n)
  }
  
  keys <- vector("list", length(tokens) - n)
  for (i in seq_len(length(tokens) - n)) {
    key <- tokens[i:(i + n - 1)]
    next_word <- tokens[i + n]
    keys[[i]] <- list()
    keys[[i]]$key <- key
    keys[[i]]$next_word <- next_word
  }
  
  table <- list()
  for (pair in keys) {
    key_str <- paste(pair$key, collapse = " ")
    if (!key_str %in% names(table)) table[[key_str]] <- c()
    table[[key_str]] <- c(table[[key_str]], pair$next_word)
  }
  
  if (is.null(start)) {
    start <- sample(names(table), 1)
  }
  
  words <- unlist(strsplit(start, " "))
  
  for (i in 1:length) {
    key_str <- paste(tail(words, n), collapse = " ")
    if (key_str %in% names(table)) {
      next_word <- sample(table[[key_str]], 1)
    } else {
      break
    }
    words <- c(words, next_word)
  }
  
  return(paste(words, collapse = " "))
}


## Question 2
#### For this question, set `seed=2025`.
#### a) Test your model using a text file of [Grimm's Fairy Tails](https://www.gutenberg.org/cache/epub/2591/pg2591.txt)
#### i) Using n=3, with the start word(s) "the king", with length=15. 
#### ii) Using n=3, with no start word, with length=15.

In [38]:
getwd()
list.files()      
lines <- readLines("grimm.txt", warn = FALSE)
head(lines, 20)   
length(lines)      
grimm_text <- paste(lines, collapse = " ")
tokens <- unlist(strsplit(tolower(grimm_text), "\\s+"))
tokens <- tokens[tokens != ""]

length(tokens)     
head(tokens, 20)
# Remove headers and empty lines
start_line <- grep("\\*\\*\\* START OF", lines)
if(length(start_line) > 0) lines <- lines[(start_line+1):length(lines)]

end_line <- grep("\\*\\*\\* END OF", lines)
if(length(end_line) > 0) lines <- lines[1:(end_line-1)]

lines <- lines[lines != ""]
grimm_text <- paste(lines, collapse = " ")
tokens <- unlist(strsplit(tolower(grimm_text), "\\s+"))
tokens <- tokens[tokens != ""]
length(tokens)  # must now be large


[1] "/home/jovyan"

[1] "apt.txt"                "armour.txt"             "BIOS512_HW10 (1).ipynb"
 [4] "BIOS512_HW3 (4).ipynb"  "BIOS512_HW4.ipynb"      "BIOS512_HW5.ipynb"     
 [7] "BIOS512_HW6 (1).ipynb"  "BIOS512_HW7 (1).ipynb"  "BIOS512_HW8 (1).ipynb" 
[10] "BIOS512_HW9.ipynb"      "cars.csv"               "ClassNotes.md"         
[13] "demographics.csv"       "diabetes.csv"           "full.csv"              
[16] "grimm.txt"              "hello.R"                "Homework2.md"          
[19] "hospitals.csv"          "housing.csv"            "HW2"                   
[22] "HW2.md"                 "install.R"              "patient_names.csv"     
[25] "README.md"              "runtime.txt"            "tng.csv"               
[28] "treatment_info.csv"     "voltages_df.csv"        "wine.csv"

[1] "The Project Gutenberg eBook of Grimms' Fairy Tales"                      
 [2] "    "                                                                    
 [3] "This ebook is for the use of anyone anywhere in the United States and"   
 [4] "most other parts of the world at no cost and with almost no restrictions"
 [5] "whatsoever. You may copy it, give it away or re-use it under the terms"  
 [6] "of the Project Gutenberg License included with this ebook or online"     
 [7] "at www.gutenberg.org. If you are not located in the United States,"      
 [8] "you will have to check the laws of the country where you are located"    
 [9] "before using this eBook."                                                
[10] ""                                                                        
[11] "Title: Grimms' Fairy Tales"                                              
[12] ""                                                                        
[13] "Author: Jacob Grimm"                                                     
[14] "        Wilhelm Grimm"                                                   
[15] ""                                                                        
[16] "Release date: April 1, 2001 [eBook #2591]"                               
[17] "                Most recently updated: June 28, 2021"                    
[18] ""                                                                        
[19] "Language: English"                                                       
[20] ""

[1] 9615

[1] 104151

[1] "the"       "project"   "gutenberg" "ebook"     "of"        "grimms'"  
 [7] "fairy"     "tales"     "this"      "ebook"     "is"        "for"      
[13] "the"       "use"       "of"        "anyone"    "anywhere"  "in"       
[19] "the"       "united"

[1] 101111

In [ ]:
generate_ngram_text_safe <- function(text, n = 3, start = NULL, length = 50) {
  tokens <- tolower(text)
  tokens <- gsub("[^a-z0-9'\\s]", "", tokens)
  tokens <- unlist(strsplit(tokens, "\\s+"))
  tokens <- tokens[tokens != ""]
  
  if(length(tokens) <= n) {
    warning("Text too short; returning the text itself.")
    return(paste(tokens, collapse = " "))
  }
  
  keys <- vector("list", length(tokens) - n)
  for(i in seq_len(length(tokens) - n)) {
    key <- tokens[i:(i + n - 1)]
    next_word <- tokens[i + n]
    keys[[i]] <- list()
    keys[[i]]$key <- key
    keys[[i]]$next_word <- next_word
  }
  
  table <- list()
  for(pair in keys) {
    key_str <- paste(pair$key, collapse = " ")
    if(!key_str %in% names(table)) table[[key_str]] <- c()
    table[[key_str]] <- c(table[[key_str]], pair$next_word)
  }
  
  if(is.null(start)) {
    start <- sample(names(table), 1)
  }
  
  words <- unlist(strsplit(start, " "))
  
  for(i in 1:length) {
    key_str <- paste(tail(words, n), collapse = " ")
    if(key_str %in% names(table)) {
      next_word <- sample(table[[key_str]], 1)
    } else {
      break
    }
    words <- c(words, next_word)
  }
  
  return(paste(words, collapse = " "))
}


lines <- readLines("grimm.txt", warn = FALSE)

lines <- lines[lines != ""]

grimm_text <- paste(lines, collapse = " ")


set.seed(2025)


cat("Grimm, n=3, start='the king', length=15:\n")
cat(generate_ngram_text_safe(grimm_text, n = 3, start = "the king", length = 15), "\n\n")

cat("Grimm, n=3, random start, length=15:\n")
cat(generate_ngram_text_safe(grimm_text, n = 3, start = NULL, length = 15), "\n")


#### b) Test your model using a text file of [Ancient Armour and Weapons in Europe](https://www.gutenberg.org/cache/epub/46342/pg46342.txt)
#### i) Using n=3, with the start word(s) "the king", with length=15. 
#### ii) Using n=3, with no start word, with length=15.

In [ ]:

lines_armour <- readLines("armour.txt", warn = FALSE)

lines_armour <- lines_armour[lines_armour != ""]

armour_text <- paste(lines_armour, collapse = " ")

tokens_armour <- unlist(strsplit(tolower(armour_text), "\\s+"))
tokens_armour <- tokens_armour[tokens_armour != ""]
length(tokens_armour)  # should be > 3

set.seed(2025)


cat("Armour, n=3, start='the king', length=15:\n")
cat(generate_ngram_text_safe(armour_text, n = 3, start = "the king", length = 15), "\n\n")

cat("Armour, n=3, random start, length=15:\n")
cat(generate_ngram_text_safe(armour_text, n = 3, start = NULL, length = 15), "\n")


#### c) Explain in 1-2 sentences the difference in content generated from each source.

I really tried with these models but my code was not working properly. I tried cleaning the txt files, but it is just not reading the files correctly dispite counting the words properly. 

## Question 3
#### a) What is a language learning model? 
#### b) Imagine the internet goes down and you can't run to your favorite language model for help. How do you run one locally?

a) A language learning model is a model that predicts patterns based on lagnuage. This is done by learning sequences from a pre-existing dataset. 

b) You can run a langauge model without internet by downloading it on your machine and providing it text to analyze.

## Question 4
#### Explain what the following vocab words mean in the context of typing `mkdir project` into the command line. If the term doesn't apply to this command, give the definition and/or an example.
| Term | Meaning |  
|------|---------|
| **Shell** | The program that interprets and excutes the instructions from the command line |
| **Terminal emulator** | The program that providers the command-line on your machine |
| **Process** | the process of running a program |
| **Signal** | ie. "terminate" |
| **Standard input** | the input stream for a program |
| **Standard output** | the output stream for a program |
| **Command line argument** | the extra information on the commandline with a command is run |
| **The environment** | the settings that impact how the process runs |

## Question 5
#### Consider the following command `find . -iname "*.R" | xargs grep read_csv`.
#### a) What are the programs?
#### b) Explain what this command is doing, part by part.

a) find and grep are the programs.

b) find. -iname "*.R" is searching for files that end in ".R". Next the "|" pipes the find output as input into xargs. Finally xargs grep read_csv runs the "grep read_csv" on files that were found and prints lines containing read_csv.

## Question 6
#### Install Docker on your machine. See [here](https://github.com/rjenki/BIOS512/blob/main/lecture18/docker_install.md) for instructions. 
#### a) Show the response when you run `docker run hello-world`.
#### b) Access Rstudio through a Docker container. Set your password and make sure your files show up on the Rstudio server. Type the command and the output you get below.
#### c) How do you log in to the RStudio server?

a) Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (arm64v8)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/


b)latest: Pulling from rocker/rstudio
3665120d345d: Pull complete 
91ed5b86de88: Pull complete 
5d246ec925db: Pull complete 
e2804bef35e8: Pull complete 
5b219f62ce36: Pull complete 
08e74fd5985d: Pull complete 
abd0190d83fb: Pull complete 
39038e16d1ba: Pull complete 
bc9245ceaac5: Pull complete 
664fb1818bbb: Pull complete 
191985778909: Pull complete 
9c1a4a0706b7: Pull complete 
a730ff463d58: Pull complete 
b8a35db46e38: Pull complete 
2c9ba66d5dbe: Pull complete 
2034506aa72f: Pull complete 
bcce866b1806: Pull complete 
cc9c938c1f51: Pull complete 
Digest: sha256:9f85211a666fb426081a6f5a01f9f9f51655262258419fa21e0ce38a5afc78d8
Status: Downloaded newer image for rocker/rstudio:latest
edb6784d2e45b3b1dfc4da7a7c4c5d093cc29641b4ebb0c57856a74006329d46

c) 1. opened http://localhost:8787; 2. entered rstudio as username and my set password as a password.